In [ ]:
#!/usr/bin/env python

from fanuc_pcdk_client import FanucPCDKClient
from robot_loc_world import RobotLocWorld
from robot_loc_joint import RobotLocJoint
from time import sleep
from copy import deepcopy
import sys

ROBOT_SPEED = 30
ROBOT_SPEED_LINEAR = 45

In [ ]:
# Robot init
robot = FanucPCDKClient()
robot.connect()

# Run teach pendant program.
robot.run_program('PCDK')
robot.reset_alarms()

# Adjust the speed of the robot
robot.set_speed(ROBOT_SPEED)

In [10]:
valid_letters = ['s', 'l', 'o', 'w', 'u', 'b']

In [ ]:
# MOVE ROBOT TO STARTING WRITING POSE (PEN TO PAPER POSE) AND RUN THIS CELL. 
# THEN USE THE INFORMATION TO FILL OUR THE 'write_start_pose' and 'write_start_pose_joints' 
# VARIABLES IN THE NEXT CELL 

# Retrieve the current world XYZWPR coordinates of the robot
loc_world = robot.get_position("world")
print(f'Current world location: {loc_world}\n')

# Retrieve the current joint coordinates of the robot
loc_joint = robot.get_position("joint")
print(f'Current joint location: {loc_joint}\n')

In [1]:
# Trajectories/offsets for each letter

# write start location
# SET POSE ACCORDINLY
write_start_pose = RobotLocWorld(x=345.437, y=128.544, z=-119.671,
                                 w=179.499, p=0.827, r=2.541, 
                                 f=False, u=True, t=True)
# SET JOINT ANGLES ACCORDINGLY
write_start_pose_joints = RobotLocJoint(j1=20, j2=0, j3=-15, j4=0, j5=-25, j6=0)

# Pen location # maybe save the joint angles for this one
pen_loc = None
# Pick up pen traj
pen_traj = [pen_loc, 'pick', pen_loc, write_start_pose]

# Return pen traj - Not actually needed

down_z = write_start_pose.z
up_z = write_start_pose.z + 10 # rises 1cm

letter_offset = (0, -50)

# tuples in (+/- x, +/- y)
# S traj
s_offset = ['down', (0, -30), (0, 30), (30, 0), (0, -30), (30, 0), (0, 30), 'up', (-60, 0)]

# L traj
l_offset = ['down', (60, 0), (0, -30), 'up', (0, 30), (-60, 0)]

# O traj
o_offset = ['down', (60, 0), (0, -30), (-60, 0), (0, 30), 'up']

# U traj
u_offset = ['down', (60, 0), (0, -30), (-60, 0), 'up', (0, 30)]

# W traj
w_offset = ['down', (60, 0), (0, -15), (-30, 0), (30, 0), (0, -15), (-60, 0), 'up', (0, 30)]

# W traj alt
w_offset = ['down', (60, -7.5), (-30, -7.5), (30, -7.5), (60, -7.5), 'up', (0, 30)]

# B traj
b_offset = ['down', (60, 0), (0, -30), (-30, 0), (0, -15), (0, 15), (-30, 0), (0, 30), 'up']

offsets = {'s': s_offset, 'l': l_offset, 'o': o_offset, 'w': w_offset, 'u': u_offset, 'b': b_offset}

# trajectories
trajectories = {'s': [], 'l': [], 'o': [], 'w': [], 'u': [], 'b': []}

In [8]:
def compute_trajectory(pose, letter):
    traj = []
    target_offsets = offsets[letter.lower()]
    curr_pose = deepcopy(pose)
    for offset in target_offsets:
        if(offset == 'down'):
            curr_pose.z = down_z
        elif(offset == 'up'):
            curr_pose.z = up_z
        elif(type(offset) == tuple):
            curr_pose.x += offset[0]
            curr_pose.y += offset[1]
        goal = deepcopy(curr_pose)
        traj.append(goal)
    return traj

In [9]:
def pickup_pen():
    # Use to complete pen pick trajectory
    for waypoint in pen_traj:
        if(type(waypoint) == RobotLocJoint):
            robot.move_joint(waypoint)
        elif(type(waypoint) == RobotLocWorld):
            robot.move(waypoint, move_linear=True, speed=ROBOT_SPEED_LINEAR)
        elif(waypoint == 'pick'):
            cur_pose = robot.get_position("world")
            curr_pose.z -= ABOVE_PEN_HEIGHT
            robot.move(curr_pose, move_linear=True, speed=ROBOT_SPEED_LINEAR)
            robot.close_gripper()
            curr_pose.z += 2*ABOVE_PEN_HEIGHT # CHANGE ACCORDINLY
            robot.move(curr_pose, move_linear=True, speed=ROBOT_SPEED_LINEAR)

In [25]:
def execute_writing(word):
    print("\n\n===============    STARTING WRITING    ===============")
    
    pickup_pen()
    
    for letter in word:
        curr_pose = robot.get_position("world")
        trajectory = compute_trajectory(robot.get_position("world"), letter)
        
        for waypoint in trajectory:
            robot.move(waypoint, move_linear=True, speed=ROBOT_SPEED_LINEAR)
    

In [26]:
terminate = False
user_input = 0
started = False
while not terminate:
    try:
        if(not started):
            print("===============  WELCOME TO THE ROBOTIC (LIMTIED) TYPEWRITER  ===============\n")
            started = True
            
        user_input = (raw_input("What 3-letter word would you like to write? (press q to quit)\
                        \nValid letters include S L O W U B\
                        \n> "))
        
        if(not (user_input == 'q')):
            if (len(user_input) > 3):
                raise ValueError
                
            for letter in user_input:
                if(letter.lower() not in valid_letters):
                    raise ValueError
                
        if(user_input == 'q'):
            print("\n===============    TERMINATING WRITING    ===============")
            terminate = True
            break
        else:
            '''
            ACTUAL WRITING STUFF GOES HERE
            '''
            execute_writing(user_input)
            
    except ValueError:
        print("Invalid word, please try again")

===============  WELCOME TO THE ROBOTIC (LIMTIED) TYPEWRITER  ===============

What 3-letter word would you like to write? (press q to quit)                        
Valid letters include S L O W U B                        
> q

===============    TERMINATING WRITING    ===============
